In [1]:
# Import required libraries
import json
import os
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter
import statistics

# Import RushDB SDK
try:
    from rushdb import RushDB
    print("✅ RushDB imported successfully")
except ImportError:
    print("❌ RushDB not found. Install with: pip install rushdb")
    raise


# Import data analysis libraries
try:
    import pandas as pd
    import numpy as np
    print("✅ Pandas and NumPy available for data analysis")
except ImportError:
    print("⚠️  Pandas/NumPy not available (optional)")

print("🏠 Real Estate Property-Buyer Matching System ready!")
print("🎯 Focus: Finding best buyers for listed properties using historical deal patterns")

✅ RushDB imported successfully
✅ Pandas and NumPy available for data analysis
🏠 Real Estate Property-Buyer Matching System ready!
🎯 Focus: Finding best buyers for listed properties using historical deal patterns


In [3]:
from dotenv import load_dotenv

load_dotenv()

RUSHDB_API_KEY = os.getenv("RUSHDB_API_KEY")

# Check RushDB API key
if RUSHDB_API_KEY == "your_rushdb_key_here":
    print("⚠️  Please set your RushDB API key!")
    print("   Get your key at: https://app.rushdb.com")
else:
    try:
        # Initialize RushDB client
        db = RushDB(RUSHDB_API_KEY)
        print("✅ Connected to RushDB successfully!")

        # Test the connection
        records = db.records.find()
        print(f"📊 Test Records request: {records}")

    except Exception as e:
        print(f"❌ RushDB connection failed: {e}")

✅ Connected to RushDB successfully!
📊 Test Records request: SearchResult(count=0, total=0)


In [4]:
print("🏠 Loading real estate datasets...")

data_dir = Path("data")

try:
    with open(data_dir / "properties.json") as f:
        all_properties = json.load(f)
    with open(data_dir / "deals.json") as f:
        deals_data = json.load(f)

except FileNotFoundError as e:
    print(f"❌ Data file not found: {e}")
    print("   Make sure properties.json and deals.json are in the 'data/' folder")
    raise

print("\n🚀 Starting data ingestion into RushDB...")

try:
    # Cleanup
    db.records.delete({})
    
    import_options = {"suggestTypes": True, "returnResult": True, "capitalizeLabels": True, "convertNumericValuesToNumbers": True}
    print("Importing listed properties...")
    properties = db.records.create_many(
        label="PROPERTY",
        data=all_properties,
        options=import_options
    )

    print("Importing completed deals...")
    deals = db.records.create_many(
        label="DEAL",
        data=deals_data,
        options=import_options
    )

except Exception as e:
    print(f"❌ Error during data ingestion: {e}")
    raise

print("\n🎉 Real estate data import complete!")

🏠 Loading real estate datasets...

🚀 Starting data ingestion into RushDB...
Importing listed properties...
Importing completed deals...

🎉 Real estate data import complete!


In [6]:
lised_properties = db.records.find({
     "labels": [
        "PROPERTY"
    ],
    "where": {
        "status": "listed",
        "state": "CA"
    },
    "limit": 10
})

results = []

for listing in lised_properties:

    price = listing['price']
    min_price = price * 0.9
    max_price =  price * 1.1
    
    # Find best fitting potential buyers
    # BUYER -> DEAL -> PROPERTY to be matched
    buyers = db.records.find({
        "labels": [
            "BUYER"
        ],
        "where": {
            "DEAL": {
                "PROPERTY": {
                    "state": listing['state'],
                    "price": { "$gte": min_price, "$lte": max_price }
                },
            }
        },
        "limit": 1000
    })
    print(f"📊 Identified {len(buyers)} potential buyers")
    
    results.append({
        'listing_id':        listing['__id'],
        'state':             listing['state'],
        'price':             price,
        'min_price':         min_price,
        'max_price':         max_price,
        'buyer_count':       len(buyers),
        'potential_buyers':  buyers
    })
    
print(f"📊 Identified {len(results)} potential offerings")


📊 Identified 4 potential buyers
📊 Identified 9 potential buyers
📊 Identified 7 potential buyers
📊 Identified 12 potential buyers
📊 Identified 10 potential buyers
📊 Identified 1 potential buyers
📊 Identified 6 potential buyers
📊 Identified 7 potential buyers
📊 Identified 2 potential buyers
📊 Identified 5 potential buyers
📊 Identified 10 potential offerings


In [7]:
import pandas as pd
from IPython.display import display, Markdown

summary_df = pd.DataFrame(
    [{k: r[k] for k in ['listing_id','state','price','min_price','max_price','buyer_count']} 
     for r in results]
)
display(summary_df)

details = []
for r in results:
    for b in r['potential_buyers']:
        details.append({
            'listing_id': r['listing_id'],
            'buyer_id':   b['__id'],
            'buyer_name': b['name'],
            'buyer_email': b['email'],
            'buyer_phone': b['phone'],
        })

buyers_df = pd.DataFrame(details)
display(buyers_df)

,listing_id,state,price,min_price,max_price,buyer_count
0,0198663c-21db-75d7-b4f3-8c9af948f563,CA,1137858,1024072.2,1251643.8,4
1,0198663c-21db-75d7-b4f3-8c640d8c9d94,CA,1816293,1634663.7,1997922.3,9
2,0198663c-21da-7a20-b23a-60275a045307,CA,1866137,1679523.3,2052750.7,7
3,0198663c-21da-7a20-b23a-6003cd4eab4a,CA,1696313,1526681.7,1865944.3,12
4,0198663c-21da-7a20-b23a-5ff1b0e2a209,CA,1794573,1615115.7,1974030.3,10
5,0198663c-21d9-76d4-a5a4-d297ee22b33a,CA,138336,124502.4,152169.6,1
6,0198663c-21d9-76d4-a5a4-d23d0081002d,CA,1347528,1212775.2,1482280.8,6
7,0198663c-21d9-76d4-a5a4-d1e370573c24,CA,1886172,1697554.8,2074789.2,7
8,0198663c-21d8-75f6-934f-fc039545e57d,CA,212750,191475.0,234025.0,2
9,0198663c-21d8-75f6-934f-fb853bcde303,CA,1247766,1122989.4,1372542.6,5


,listing_id,buyer_id,buyer_name,buyer_email,buyer_phone
0,0198663c-21db-75d7-b4f3-8c9af948f563,0198663c-316c-7abb-8ce6-8987955d9bce,Marco Turcotte II,Kay.Hodkiewicz@hotmail.com,859-306-9712 x156
1,0198663c-21db-75d7-b4f3-8c9af948f563,0198663c-3166-7dae-b5d1-63d5b34fc6a4,Dianna Keeling,Tod.Russel@hotmail.com,902.209.6974 x972
2,0198663c-21db-75d7-b4f3-8c9af948f563,0198663c-312c-7bc0-840b-af91e8580434,Rita Altenwerth,Ruthie_Bode@yahoo.com,577.852.3420
3,0198663c-21db-75d7-b4f3-8c9af948f563,0198663c-312a-7933-9b1b-a9ec6f52e083,Benny Wunsch,Stewart.Hintz@hotmail.com,1-582-586-9199
4,0198663c-21db-75d7-b4f3-8c640d8c9d94,0198663c-3179-7ca4-877a-5daf68727222,Mrs. Lucia Conroy,Arianna.Larson17@yahoo.com,352.556.4088
...,...,...,...,...,...
58,0198663c-21d8-75f6-934f-fb853bcde303,0198663c-317b-7987-a013-b1f6b552facc,Hope Bauch,Rhiannon.Denesik@hotmail.com,387-982-2113 x601
59,0198663c-21d8-75f6-934f-fb853bcde303,0198663c-317a-7e22-b44e-86b01428ae97,Brenda Denesik Jr.,Alysha.Hermiston45@hotmail.com,610-880-6620 x09188
60,0198663c-21d8-75f6-934f-fb853bcde303,0198663c-316a-7dda-97dd-0d76dc05bea2,Beth Torphy MD,Ambrose_Brekke@hotmail.com,1-350-332-0543 x9898
61,0198663c-21d8-75f6-934f-fb853bcde303,0198663c-316a-7dda-97dd-0d347467e6fc,Julia Bradtke,Ron_Thompson68@yahoo.com,1-228-397-9009 x1501
